In [139]:
import pandas as pd
from geopy.distance import geodesic
from skspatial.objects import LineSegment
from typing import List, Tuple

In [2]:
df = pd.read_json("links.json")

In [199]:
def get_closest_distance_between_segments(ls1, ls2):
    a0, a1 = np.array([(x[0], x[1], 0) for x in ls1])
    b0, b1 = np.array([(x[0], x[1], 0) for x in ls2])
    
    # Calculate denomitator
    A = a1 - a0
    B = b1 - b0
    magA = np.linalg.norm(A)
    magB = np.linalg.norm(B)
    
    _A = A / magA
    _B = B / magB
    
    cross = np.cross(_A, _B);
    denom = np.linalg.norm(cross)**2
    
    
    # If lines are parallel (denom=0) test if lines overlap.
    # If they don't overlap then there is a closest point solution.
    # If they do overlap, there are infinite closest positions, but there is a closest distance
    if not denom:
        d0 = np.dot(_A,(b0-a0))
        d1 = np.dot(_A,(b1-a0))
            
        # Is segment B before A?
        if d0 <= 0 >= d1:
            if np.absolute(d0) < np.absolute(d1):
                return a0[:2],b0[:2],np.linalg.norm(a0-b0)
            return a0[:2],b1[:2],np.linalg.norm(a0-b1)
                
                
        # Is segment B after A?
        elif d0 >= magA <= d1:
            if np.absolute(d0) < np.absolute(d1):
                return a1[:2],b0[:2],np.linalg.norm(a1-b0)
            return a1[:2],b1[:2],np.linalg.norm(a1-b1)
    
        # Segments overlap, return distance between parallel segments
        return None, None, np.linalg.norm(((d0*_A)+a0)-b0)
    
    # Lines criss-cross: Calculate the projected closest points
    t = (b0 - a0);
    detA = np.linalg.det([t, _B, cross])
    detB = np.linalg.det([t, _A, cross])

    t0 = detA/denom;
    t1 = detB/denom;

    pA = a0 + (_A * t0) # Projected closest point on segment A
    pB = b0 + (_B * t1) # Projected closest point on segment B

    # Clamp projections
    if t0 < 0:
            pA = a0
    elif t0 > magA:
            pA = a1
        
    if t1 < 0:
            pB = b0
    elif t1 > magB:
            pB = b1
            
    # Clamp projection A
    if (t0 < 0) or (t0 > magA):
            dot = np.dot(_B,(pA-b0))
            if dot < 0:
                dot = 0
            elif dot > magB:
                dot = magB
            pB = b0 + (_B * dot)
    
    # Clamp projection B
    if (t1 < 0) or (t1 > magB):
            dot = np.dot(_A,(pB-a0))
            if dot < 0:
                dot = 0
            elif dot > magA:
                dot = magA
            pA = a0 + (_A * dot)

    
    return pA[:2], pB[:2], np.linalg.norm(pA - pB)

In [234]:
def polyline_to_line_segments(polyline) -> List[Tuple[float, float]]:
    segments = []
    for i in range(0, len(polyline) - 1):
        coord1, coord2 = polyline[i], polyline[i+1]
        segments.append((coord1, coord2))
    return segments
        
        
def is_polyline_segment_close(polyline1, polyline2, threshold_m=10):
    """
    Polyline format: [(lon, lat), (lon, lat), ...]
    """

    line_segments_1 = polyline_to_line_segments(polyline1)
    line_segments_2 = polyline_to_line_segments(polyline2)

    for lseg1 in line_segments_1:
        for lseg2 in line_segments_2:
            c1, c2, dist = get_closest_distance_between_segments(lseg1, lseg2)
            if c1 is not None and c2 is not None:
                if not np.isnan(c1[0]):
                    actual_dist = geodesic(c1[::-1], c2[::-1])
                    # print("Distance", actual_dist.m)
                    if actual_dist.m <= threshold_m:
                        return True
            # if dist <= threshold_deg:
                # return True

    return False

In [235]:
is_polyline_segment_close(df.iloc[0].polyline, df.iloc[1].polyline)

True

In [250]:
from itertools import combinations
for (_, r1), (_, r2) in combinations(df.iterrows(), 2):
    if is_polyline_segment_close(r1.polyline, r2.polyline):
        print("Close %s with %s" % (r1.link_id, r2.link_id))

Close db864ee5-da16-4ef6-b0b1-daf4640a5fe8 with eb762842-8cfa-451d-bcc5-792edcc79b90


/var/folders/71/711bc3rd5zx9ngj8wtlnwjjw0000gn/T/ipykernel_4176/2483369643.py:12: RuntimeWarning: invalid value encountered in divide
  _B = B / magB
/var/folders/71/711bc3rd5zx9ngj8wtlnwjjw0000gn/T/ipykernel_4176/2483369643.py:11: RuntimeWarning: invalid value encountered in divide
  _A = A / magA


Close db864ee5-da16-4ef6-b0b1-daf4640a5fe8 with 6535b836-4248-489c-9e55-11b6ebbc9780
Close eb762842-8cfa-451d-bcc5-792edcc79b90 with 6535b836-4248-489c-9e55-11b6ebbc9780
Close 66370a35-abc9-454e-8a0c-fc64fc3bd7d1 with 9019edd9-3401-4620-965b-9e76f4dc776e
Close 773f7adf-296b-41e7-9086-da97521ba353 with 097575e9-c567-428b-8629-bc420b97cb74
Close 773f7adf-296b-41e7-9086-da97521ba353 with 83de80ef-4a9d-4d35-b7d6-c6fab2e78ed3
Close 27b20ef6-adbc-462e-97d9-bd822d2040dd with 9eafdeb9-4358-45ed-8e43-abaf91ef7327
Close 27b20ef6-adbc-462e-97d9-bd822d2040dd with 72677f82-dc41-4a71-8d24-5ace4868c87b
Close 7f6e0190-94b3-4848-a3b8-f8d89d507eab with edc45797-74d3-4fd6-b8b2-e8d91331a0dd
Close 7f6e0190-94b3-4848-a3b8-f8d89d507eab with d0f8a8b8-336b-47c6-869a-7bc51b700ff0
Close bb6bbacd-efc4-4695-a2f4-29225a433cf9 with 3aac7e76-d03e-483a-9ad3-81517a5f3d90
Close bb6bbacd-efc4-4695-a2f4-29225a433cf9 with 832e4bb7-d56b-4082-a214-1a2d7b240a30
Close a8516b27-0861-4e4b-b873-b31ec4b4a330 with b6505f80-83ca-48e

KeyboardInterrupt: 